In [5]:
import polars as pl
import pandas as pd 
import numpy as np 
import plotly.graph_objects as go 
import gc 

In [6]:
df1_pl = pl.from_pandas(pd.read_excel("input/20250715-ketquathi-ct2018a.xlsx", sheet_name="Sheet1").astype(str))
df2_pl = pl.from_pandas(pd.read_excel("input/20250715-ketquathi-ct2018a.xlsx", sheet_name="Sheet2").astype(str))

df = pl.concat([df1_pl, df2_pl], how="vertical").lazy()
df = df.with_columns([
    pl.col("Toán").str.replace("nan", "0").alias("Toán"),
    pl.col("Văn").str.replace("nan", "0").alias("Văn"),
    *[
        pl.when(pl.col(col) == "nan").then(None).otherwise(pl.col(col)).alias(col)
        for col in df.columns
        if col not in ["Toán", "Văn"]
    ]
])

del df1_pl, df2_pl
gc.collect()
print(df.head().collect())


C:\Users\musba\AppData\Local\Temp\ipykernel_12568\4052457608.py:10: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  for col in df.columns


shape: (5, 15)
┌─────┬───────────┬──────┬──────┬───┬──────┬───────────────────────────────┬───────────┬──────────────────┐
│ STT ┆ SOBAODANH ┆ Toán ┆ Văn  ┆ … ┆ Địa  ┆ Giáo dục kinh tế và pháp luật ┆ Ngoại ngữ ┆ Mã môn ngoại ngữ │
│ --- ┆ ---       ┆ ---  ┆ ---  ┆   ┆ ---  ┆ ---                           ┆ ---       ┆ ---              │
│ str ┆ str       ┆ str  ┆ str  ┆   ┆ str  ┆ str                           ┆ str       ┆ str              │
╞═════╪═══════════╪══════╪══════╪═══╪══════╪═══════════════════════════════╪═══════════╪══════════════════╡
│ 1   ┆ 1000001   ┆ 5.75 ┆ 7.75 ┆ … ┆ null ┆ null                          ┆ null      ┆ null             │
│ 2   ┆ 1000002   ┆ 8.0  ┆ 8.25 ┆ … ┆ null ┆ null                          ┆ null      ┆ null             │
│ 3   ┆ 1000003   ┆ 6.75 ┆ 8.5  ┆ … ┆ null ┆ null                          ┆ null      ┆ null             │
│ 4   ┆ 1000004   ┆ 5.25 ┆ 7.5  ┆ … ┆ null ┆ null                          ┆ null      ┆ null             │
│ 5   ┆ 10000

In [7]:
df.collect().height

1131136

In [8]:
NGOAI_NGU = {
    "N1" : "Anh", 
    "N2" : "Nga", 
    "N3" : "Pháp", 
    "N4" : "Trung",
    "N5" : "Đức", 
    "N6" : "Nhật", 
    "N7" : "Hàn"
}

ngoai_ngu_table = (
    df
    .select(["SOBAODANH", "Mã môn ngoại ngữ", "Ngoại ngữ"])
    .filter(
        (pl.col("Ngoại ngữ").is_not_null()) | (pl.col("Ngoại ngữ") != "nan") |
        (pl.col("Mã môn ngoại ngữ").is_not_null()) | (pl.col("Ngoại ngữ") != "nan")
    )
    .with_columns(
        pl.col("Mã môn ngoại ngữ").replace(NGOAI_NGU).alias("Mã môn ngoại ngữ")
    )
    .collect()
    .pivot(
        on = "Mã môn ngoại ngữ",
        index = "SOBAODANH", 
        values = "Ngoại ngữ",
    )
).lazy()

ngoai_ngu_table.collect().head()

SOBAODANH,Anh,Nhật,Trung,Hàn,Đức,Pháp,Nga
str,str,str,str,str,str,str,str
"""1000553""","""7.5""",null,null,null,null,null,null
"""1000555""","""8.75""",null,null,null,null,null,null
"""1000556""","""7.0""",null,null,null,null,null,null
"""1000557""","""8.75""",null,null,null,null,null,null
"""1000558""","""7.5""",null,null,null,null,null,null


In [9]:
result_df = df.drop(["Ngoại ngữ", "Mã môn ngoại ngữ"]).join(ngoai_ngu_table,on="SOBAODANH",how="full").with_columns(
    pl.col("SOBAODANH").str.slice(0, 2).alias("Mã tỉnh thành")
)
result_df.collect().head()

STT,SOBAODANH,Toán,Văn,Lí,Hóa,Sinh,Tin học,Công nghệ công nghiệp,Công nghệ nông nghiệp,Sử,Địa,Giáo dục kinh tế và pháp luật,SOBAODANH_right,Anh,Nhật,Trung,Hàn,Đức,Pháp,Nga,Mã tỉnh thành
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""1""","""1000001""","""5.75""","""7.75""",null,"""7.75""","""8.25""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""10"""
"""2""","""1000002""","""8.0""","""8.25""","""8.5""","""6.75""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""10"""
"""3""","""1000003""","""6.75""","""8.5""","""8.75""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""10"""
"""4""","""1000004""","""5.25""","""7.5""","""6.5""","""5.5""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""10"""
"""5""","""1000005""","""0""","""7.0""",null,null,null,null,null,null,"""5.5""","""6.25""",null,null,null,null,null,null,null,null,null,"""10"""


In [10]:
result_df.collect().write_csv("output/20250715-ketquathi-ct2018a.csv")

In [11]:
result_df.collect().write_parquet("output/20250715-ketquathi-ct2018a.parquet")